# Net Spider for HTML Pages

In [6]:
import sqlite3 as sql # for databases
from bs4 import BeautifulSoup # web scraping tool
import requests
import json

### Soup object

In [ ]:
targetWikiPage = "https://en.wikipedia.org/wiki/List_of_highest-grossing_films" # wikipedia page url
baseWikiPage = "https://en.wikipedia.org" # base url for accessing additional pages

def get_soup(url: str) -> BeautifulSoup:
    response = requests.get(url)
    return BeautifulSoup(response.text, "html.parser")

# Get the soup object for the target Wikipedia page
soup = get_soup(targetWikiPage)

#### Parsing data from source

In [ ]:
# Find all tables with class "wikitable" on the page
wiki_tables = soup.find_all("table", class_="wikitable")

# We are interested in the first table
highest_grossing_films_table = wiki_tables[0]

# Get all rows from the table
table_rows = highest_grossing_films_table.find_all("tr")

# Initialize an empty list to store film data
film_data = []

# Iterate over each row, skipping the header row
for row in table_rows[1:]:
    # Get all cells and headers in the row
    cells = row.find_all("td")
    headers = row.find_all("th")
    
    # Get the film link from the header
    film_link = headers[0].find_all("a")[0]
    
    # Create a dictionary to store film details
    film_details = {
        "year": cells[3].text.strip(),  # Release year of the film
        "title": film_link.text.strip(),  # Title of the film
        "revenue": int("".join([char for char in cells[2].text.strip() if char.isnumeric()])),  # Box office revenue
        "href": film_link["href"],  # Link to the film's Wikipedia page
    }

    # Add the film details to our list
    film_data.append(film_details)

# Display the collected film data
film_data

[{'year': '2009',
  'title': 'Avatar',
  'revenue': 2923706026,
  'href': '/wiki/Avatar_(2009_film)'},
 {'year': '2019',
  'title': 'Avengers: Endgame',
  'revenue': 2797501328,
  'href': '/wiki/Avengers:_Endgame'},
 {'year': '2022',
  'title': 'Avatar: The Way of Water',
  'revenue': 2320250281,
  'href': '/wiki/Avatar:_The_Way_of_Water'},
 {'year': '1997',
  'title': 'Titanic',
  'revenue': 2257844554,
  'href': '/wiki/Titanic_(1997_film)'},
 {'year': '2015',
  'title': 'Star Wars: The Force Awakens',
  'revenue': 2068223624,
  'href': '/wiki/Star_Wars:_The_Force_Awakens'},
 {'year': '2018',
  'title': 'Avengers: Infinity War',
  'revenue': 2048359754,
  'href': '/wiki/Avengers:_Infinity_War'},
 {'year': '2025',
  'title': 'Ne Zha 2',
  'revenue': 1985082560,
  'href': '/wiki/Ne_Zha_2'},
 {'year': '2021',
  'title': 'Spider-Man: No Way Home',
  'revenue': 1922598800,
  'href': '/wiki/Spider-Man:_No_Way_Home'},
 {'year': '2024',
  'title': 'Inside Out 2',
  'revenue': 1698863816,
  'h

#### Extracting data from HTML

In [12]:
# We do a lot of consecutive requests, so we need to handle cases when the request fails
def retry(max_retries: int = 3):
    def decorator(func):
        def wrapper(*args, **kwargs):
            attempts = 0  # Number of attempts made
            while attempts < max_retries:
                try:
                    return func(*args, **kwargs)
                except Exception:
                    attempts += 1  # Increment attempts if an exception occurs
            raise Exception  # Raise exception if max retries reached

        return wrapper

    return decorator

@retry()
def get_country(href: str) -> str:
    response = requests.get(baseWikiPage + href)
    soup = BeautifulSoup(response.text, "html.parser")
    data = soup.select_one("table", class_="infobox")
    try:
        country = data.find("th", string="Country").find_next("td").text.strip()
    except Exception:
        country_td = data.find("th", string="Countries").find_next("td")
        ul = country_td.find("ul")
        country = (
            ", ".join([li.text.strip() for li in ul.find_all("li")])
            if ul
            else country_td.text.strip()
        )
    return country
# This function will parse the film page and extract director data
@retry()
def get_director(film_href: str) -> str:
    response = requests.get(baseWikiPage + film_href)  # Get the film page
    soup = BeautifulSoup(response.text, "html.parser")  # Parse the page with BeautifulSoup
    infobox = soup.select_one("table", class_="infobox")  # Find the infobox table
    director_td = infobox.find("th", string="Directed by").find_next("td")  # Find the director cell
    director_list = director_td.find("ul")  # Check if there is a list of directors
    director = (
        ", ".join([li.text.strip() for li in director_list.find_all("li")])
        if director_list
        else director_td.text.strip()  # Get the director(s) name(s)
    )
    return director  # Return the director(s) name(s)


#### For each movie, we extract the following data:

In [16]:
# Parse film pages and extract additional data
# If our script fails to parse the page, we'll set the value to "NONE"
# and print the name of the problematic film
for movie in film_data:
    try:
        movie["country"] = get_country(movie["href"])
    except Exception:
        movie["country"] = "NONE"
        print(f"Failed to get country for {movie['title']}")

    try:
        movie["director"] = get_director(movie["href"])
    except Exception:
        movie["director"] = "NONE"
        print(f"Failed to get director for {movie['title']}")

# Fix specific film data manually
for movie in film_data:
    if movie["title"] == "Ne Zha 2":
        movie["country"] = "China"
        movie["director"] = "Jiaozi"

# Let's use SQLite to store our data
conn = sql.connect("../data/films.db")
sqlProcessor = conn.cursor()

# This command is needed to make it possible to rerun this cell
sqlProcessor.execute("DROP TABLE IF EXISTS films")

# Schema is the same as in assignment instruction
sqlProcessor.execute(
    """
    CREATE TABLE IF NOT EXISTS films (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT NOT NULL,
        release_year INTEGER,
        director TEXT,
        box_office REAL,
        country TEXT
    )
    """
)

# Insert film data one by one
for movie in film_data:
    sqlProcessor.execute(
        """
        INSERT INTO films (title, release_year, director, box_office, country)
        VALUES (?, ?, ?, ?, ?)
        """,
        (
            movie["title"],
            movie["year"],
            movie["director"],
            movie["revenue"],
            movie["country"],
        ),
    )

conn.commit()

# Check if data was inserted correctly
sqlProcessor.execute("SELECT * FROM films").fetchall()


Failed to get country for Ne Zha 2
Failed to get director for Ne Zha 2


[(1,
  'Avatar',
  2009,
  'James Cameron',
  2923706026.0,
  'United Kingdom[2], United States[2]'),
 (2,
  'Avengers: Endgame',
  2019,
  'Anthony Russo, Joe Russo',
  2797501328.0,
  'United States'),
 (3,
  'Avatar: The Way of Water',
  2022,
  'James Cameron',
  2320250281.0,
  'United States'),
 (4, 'Titanic', 1997, 'James Cameron', 2257844554.0, 'United States'),
 (5,
  'Star Wars: The Force Awakens',
  2015,
  'J. J. Abrams',
  2068223624.0,
  'United States'),
 (6,
  'Avengers: Infinity War',
  2018,
  'Anthony Russo, Joe Russo',
  2048359754.0,
  'United States'),
 (7, 'Ne Zha 2', 2025, 'Jiaozi', 1985082560.0, 'China'),
 (8,
  'Spider-Man: No Way Home',
  2021,
  'Jon Watts',
  1922598800.0,
  'United States'),
 (9, 'Inside Out 2', 2024, 'Kelsey Mann', 1698863816.0, 'United States'),
 (10,
  'Jurassic World',
  2015,
  'Colin Trevorrow',
  1671537444.0,
  'United States'),
 (11, 'The Lion King', 2019, 'Jon Favreau', 1656943394.0, 'United States'),
 (12, 'The Avengers', 2012, 

In [17]:
# Save it also as JSON
with open("../data/films.json", "w") as file:
    json.dump(film_data, file)